In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel('routine.xlsx', sheet_name=2 ,skiprows=4)
data.columns = data.columns.str.replace('\n', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(r'\.\d+', '', regex=True)
df = data.fillna('-') 
df.columns = df.columns.str.upper()

In [ ]:
data_cleaning = df.drop(["CLASS_SIZE", "ASSIGNED_HALL", "HALL_CAPACITY_MATCH", "UNNAMED_57"], axis='columns')
len(data.columns)
indexList = list(range(61))
for i in range(2, 61, 5):
    indexList.remove(i)
clean_data = data_cleaning.iloc[:, indexList]
clean_data.to_excel('data.xlsx', index=False)
clean_data.head()